In [3]:
import pickle
import pywt

from sklearn import preprocessing

import numpy as np
import collections
import threading

In [4]:
import pickle


clf=pickle.load(open("random_tree_selector.p","rb"))

In [5]:
#!/usr/bin/env python
# encoding: utf-8

## Module infomation ###
# Python (3.4.4)
# numpy (1.10.2)
# PyAudio (0.2.9)
# matplotlib (1.5.1)
# All 32bit edition
########################

import numpy as np
import pyaudio

import matplotlib.pyplot as plt

import rospy

from keyboard.msg import Key
import pickle
import uuid
import scipy.fftpack
from std_msgs.msg import String

rospy.init_node('audio_listener', anonymous=True)

some_name=str(uuid.uuid1())

import scipy.signal

class SpectrumAnalyzer:
    FORMAT = pyaudio.paFloat32
    CHANNELS = 1
    #RATE = 1000
    RATE= 16384
    CHUNK = 2048 # 1/16
    
    DECIMATED_BUFFER_SIZE=256
    #CHUNK = 1024
    START = 0
    N = 512
    #N= 1024
    ELEMENTS=25
    wave_x = 0
    wave_y = 0
    spec_x = 0
    spec_y = 0
    data = []
    xx=[]

    yy=[]

    types_of_signal=[]
    button=0
    running_buffer=collections.deque([0 for n in range(DECIMATED_BUFFER_SIZE)],
                                     maxlen=DECIMATED_BUFFER_SIZE) #1s of signal
    
    stop=False
    def __init__(self):

        self.categorizer=clf
        self.rate = rospy.Rate(10)
        self.lock = threading.Lock()
        self.pa = pyaudio.PyAudio()
        
        self.stream = self.pa.open(format = self.FORMAT,
            channels = self.CHANNELS, 
            rate = self.RATE, 
            input = True,
            output = False,
            frames_per_buffer = self.CHUNK,
            stream_callback=self.new_frame
                                  
                                  )
        self.pub = rospy.Publisher('stiffened', String, queue_size=10)
        self.loop()

    def new_frame(self, data, frame_count, time_info, status):
        data = np.fromstring(data, np.float32)
        
        with self.lock:
            self.running_buffer.extend(self.decimate_data(data))
            
            wavelet_pockets= pywt.WaveletPacket(data=self.running_buffer,
                                                         wavelet='db4',
                                                         mode='symmetric',
                                                         maxlevel=6)
            wavelet_pockets_elements=wavelet_pockets.get_level(6,'freq')[0:self.ELEMENTS]
            
            feature_vector= self.calculate_waveletpacket_features(wp_elements=wavelet_pockets_elements)
            wavelect_pocket_vector=self.calculate_waveletpacket(wp_elements=wavelet_pockets_elements)
            
            result=self.categorizer.predict(feature_vector.reshape(1, -1))
            if(result[0]==32):
                self.pub.publish("relaxed")
            elif(result[0]==13):
                self.pub.publish("stiffened")
            
            
            if self.stop:
                return None, pyaudio.paComplete
        return None, pyaudio.paContinue
    
    def decimate_data(self,audio_sig):
        decimated=scipy.signal.decimate(audio_sig,8,ftype='fir')
        decimated=scipy.signal.decimate(decimated,8,ftype='fir')
        return decimated
    def calculate_waveletpacket(self,data=None, level=6,elements=25, wp_elements=None):
        if wp_elements is None:
            wp = pywt.WaveletPacket(data=data, wavelet='db4', mode='symmetric', maxlevel=level)
            wp_elements=wp.get_level(level,'freq')[0:elements]

        #print(
        feature_vector=np.concatenate([node.data for node in wp_elements]) #create one row
        return feature_vector
    
    def calculate_waveletpacket_features(self,data= None,level=6,elements=25, wp_elements=None):
        if wp_elements is None:
            wp = pywt.WaveletPacket(data=data, wavelet='db4', mode='symmetric', maxlevel=6)
            wp_elements=wp.get_level(level,'freq')[0:elements]
        means=np.array([np.mean(node.data) for node in wp_elements])
        powers=np.array([np.linalg.norm(node.data) for node in wp_elements])
        return np.concatenate((means, powers))  
        

    def loop(self):
        try:
            while not rospy.is_shutdown():

                self.rate.sleep()
        except KeyboardInterrupt:
            self.pa.close()
        self.stop=True
        print("End...")

    def audioinput(self):
        ret = self.stream.read(self.CHUNK)
        ret = np.fromstring(ret, np.float32)
        return ret

    def fft(self):
        self.wave_x = range(self.START, self.START + self.N)
        self.wave_y = self.data[self.START:self.START + self.N]
        self.spec_x = np.fft.rfftfreq(self.N, d = 1.0 / self.RATE)  
        y = np.fft.rfft(self.data[self.START:self.START + self.N])    
        self.spec_y = np.abs(y)
        
    def graphplot(self):
        plt.clf()
        # wave
        plt.subplot(311)
        plt.plot(self.wave_x, self.wave_y)
        plt.axis([self.START, self.START + self.N, -0.5, 0.5])
        plt.xlabel("time [sample]")
        plt.ylabel("amplitude")
        #Spectrum
        plt.subplot(312)
        plt.plot(self.spec_x, self.spec_y, marker= 'o', linestyle='-')
        plt.axis([0, self.RATE / 2, 0, 50])
        plt.xlabel("frequency [Hz]")
        plt.ylabel("amplitude spectrum")
        #Pause
        #plt.pause(.01)


    

In [ ]:
spec = SpectrumAnalyzer()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:79: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
